 In this example we test the capabilities of **ReactiveMP.jl** to run smoothing algorithm for large-scale datastream.

In [1]:
using Rocket, ReactiveMP, GraphPPL, Distributions, BenchmarkTools

In [2]:
@model function smoothing(n, x0, P::ConstVariable)
    
    x_prior ~ NormalMeanVariance(mean(x0), cov(x0)) 

    x = randomvar(n)
    y = datavar(Float64, n)
    c = constvar(1.0)

    x_prev = x_prior

    for i in 1:n
        x[i] ~ x_prev + c
        y[i] ~ NormalMeanVariance(x[i], P)
        
        x_prev = x[i]
    end

    return x, y
end

In [3]:
P = 1.0

n = 10_000
data = collect(1:n) + rand(Normal(0.0, sqrt(P)), n);

In [4]:
@benchmark res = inference(
    model   = Model(smoothing, $n, NormalMeanVariance(0.0, 10000.0), $P), 
    data    = (y = $data,), 
    options = model_options(limit_stack_depth = 500)
)

BenchmarkTools.Trial: 10 samples with 1 evaluation.
 Range (min … max):  345.833 ms … 704.682 ms  ┊ GC (min … max):  0.00% … 52.96%
 Time  (median):     542.715 ms               ┊ GC (median):    33.96%
 Time  (mean ± σ):   539.657 ms ±  93.720 ms  ┊ GC (mean ± σ):  35.01% ± 13.84%

  ▁                    ▁     ▁    █▁▁▁            ▁           ▁  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁█▁▁▁▁████▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁█ ▁
  346 ms           Histogram: frequency by time          705 ms <

 Memory estimate: 223.43 MiB, allocs estimate: 4291109.